In [1]:
import copy

import pandas as pd

from constraint_relax import search_mfs
from experiment_constraint_data_processor import process_experiment_constraint_data
from item_extrator import data_path
from similarity_calculator import compute_similarity
from utils import get_df_len

In [2]:
problem_list = process_experiment_constraint_data('./experiment_constraint_data.csv')

In [3]:
sol_list = []
threshold_list = [0, 0.5, 1]
for problem in problem_list:
    prob_sol_list = []
    p1 = copy.deepcopy(problem) # original constraint
    prob_sol_list.append(p1.constraint_list)
    prob_sol_list.append(get_df_len(p1.solve()))

    for threshold in threshold_list:
        p = copy.deepcopy(problem)
        mfs = search_mfs(p.constraint_list, copy.copy(p.constraint_list), data_path)
        p.relax(mfs, threshold)
        prob_sol_list.append(p.constraint_list)
        prob_items = p.solve()
        prob_sol_list.append(get_df_len(prob_items))
        if not prob_items is None:
            prob_sol_list.append((compute_similarity(p.constraint_list, prob_items))['similarity'].mean())
        else:
            prob_sol_list.append(0)
    sol_list.append(prob_sol_list)
result = pd.DataFrame(sol_list)

In [4]:
result

,0,1,2,3,4,5,6,7,8,9,10
0,"{price: 2699.0: 10: Nature.LESS, storage: 512....",14,"{price: 2699.0: 10: Nature.LESS, os: 10: 4: Na...",15,0.720075,"{price: 2699.0: 10: Nature.LESS, storage: 512....",15,0.720075,"{camera: : 1: Nature.EQUAL, os: 10: 4: Nature....",15,0.720075
1,"{os: os: 8: Nature.EQUAL, weight: 2.0: 9: Natu...",0,"{brand: : 7: Nature.EQUAL, price: 3271.04: 6: ...",1,0.581119,"{price: 3271.04: 6: Nature.LESS, brand: : 7: N...",1,0.581119,"{os: os: 8: Nature.EQUAL, price: 3271.04: 6: N...",1,0.581119
2,"{ram: 16.0: 9: Nature.MORE, price: 2664.04: 5:...",0,"{os: 11: 4: Nature.EQUAL, weight: 1.44: 9: Nat...",0,0.000000,"{ram: 16.0: 9: Nature.MORE, os: 11: 4: Nature....",0,0.000000,"{ram: 16.0: 9: Nature.MORE, weight: 1.44: 9: N...",4,0.487346
3,"{weight: 1.2: 6: Nature.LESS, storage: 1024.0:...",0,"{price: 5547.0: 3: Nature.LESS, brand: huawei|...",0,0.000000,{brand: huawei|dell|microsoft: 2: Nature.EQUAL...,0,0.000000,"{storage: 1024.0: 7: Nature.MORE, price: : 3: ...",2,0.666667
4,"{os: 11: 4: Nature.EQUAL, cpu: ryzen 5|i7: 4: ...",5,"{cpu: ryzen 5|i7: 4: Nature.EQUAL, weight: 2.4...",6,0.639378,"{os: 11: 4: Nature.EQUAL, weight: : 1: Nature....",6,0.624030,"{weight: : 1: Nature.LESS, cpu: ryzen 5|i7: 4:...",6,0.624030
...,...,...,...,...,...,...,...,...,...,...,...
495,"{ram: 4.0: 6: Nature.MORE, storage: 1024.0: 1:...",0,"{storage: 512: 1: Nature.MORE, brand: msi|acer...",0,0.000000,"{price: 4097.0: 3: Nature.LESS, os: os: 9: Nat...",3,0.607755,"{brand: msi|acer|hp|huawei: 1: Nature.EQUAL, r...",3,0.607755
496,"{storage: 1024.0: 4: Nature.MORE, weight: 1.47...",3,"{storage: 1024.0: 4: Nature.MORE, weight: 1.47...",3,0.676014,"{ram: : 2: Nature.MORE, weight: 1.47: 7: Natur...",3,0.687003,"{weight: 1.47: 7: Nature.LESS, storage: 1024.0...",3,0.687003
497,"{camera: True: 1: Nature.EQUAL, weight: 1.44: ...",9,"{storage: 256.0: 6: Nature.MORE, camera: True:...",10,0.502405,"{os: 11: 2: Nature.EQUAL, storage: 256.0: 6: N...",30,0.435484,"{storage: 256.0: 6: Nature.MORE, os: 11: 2: Na...",10,0.407405
498,"{storage: 1024.0: 5: Nature.MORE, os: 10: 10: ...",0,"{cpu: ryzen 3|i3|ryzen 5|i5: 2: Nature.EQUAL, ...",0,0.000000,"{storage: 1024.0: 5: Nature.MORE, os: 10: 10: ...",1,0.920000,"{cpu: : 2: Nature.EQUAL, storage: 1024.0: 5: N...",1,0.920000


In [5]:
pd.value_counts(result.iloc[:, 1] == 0)

True     285
False    215
Name: 1, dtype: int64

In [6]:
valid = ((result.iloc[:, 1] <= result.iloc[:, 3]) & (result.iloc[:, 3] <= result.iloc[:, 6]) & (result.iloc[:, 6] <= result.iloc[:, 9]))
valid_count = pd.value_counts(valid)
print(valid_count)

True     478
False     22
dtype: int64


In [7]:
valid = ((result.iloc[:, 1] < result.iloc[:, 3]) & (result.iloc[:, 3] < result.iloc[:, 6]) & (result.iloc[:, 6] < result.iloc[:, 9]))
valid_count = pd.value_counts(valid)
print(valid_count)

False    498
True       2
dtype: int64


In [8]:
result[valid]

,0,1,2,3,4,5,6,7,8,9,10
158,"{os: 10: 5: Nature.EQUAL, weight: 1.91: 3: Nat...",59,"{camera: True: 3: Nature.EQUAL, weight: 2.0115...",61,0.500534,"{camera: : 3: Nature.EQUAL, os: 10: 5: Nature....",63,0.364820,"{ram: 8.0: 5: Nature.MORE, weight: : 3: Nature...",74,0.438007
482,"{os: os: 3: Nature.EQUAL, cpu: ryzen 5: 6: Nat...",0,"{storage: 512: 5: Nature.MORE, os: os|10: 3: N...",6,0.674799,"{weight: 1.52: 2: Nature.LESS, os: : 3: Nature...",8,0.595528,"{price: 5056.09: 8: Nature.LESS, ram: 8.0: 4: ...",9,0.527914
